## Cross-Platform verification

Related papers [Elben et al, PRL 2019](https://doi.org/10.1103/PhysRevLett.124.010504), [Zhu et al, Nat. Comm 2022](https://www.nature.com/articles/s41467-022-34279-5)

In [1]:
using Revise
using ProgressMeter
using ITensors
using RandomMeas

In [2]:
#Perform RM in the first experiment
N = 6
ξ = siteinds("Qubit", N)
depth = 2
circuit = random_circuit(ξ, depth)
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);
ρ1 = apply(circuit,ρ0,apply_dag=true);
println("state prepared in the `experiment' 1 ")

state prepared in the `experiment' 1 


In [3]:
#Perform Randomized measurements
nu=200 #Number of random unitaries
NM=1000 #Number of projective measurements per unitary
u = Vector{Vector{ITensor}}()
data1 = zeros(Int8,(nu,NM,N)) #Data storage
@showprogress dt=1 for r in 1:nu
            push!(u,get_rotations(ξ,1)) #generate random unitary
            data1[r,:,:] = get_RandomMeas(ρ1,u[r],NM) #data acquisation in simulated quantum device
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:43


In [4]:
p = 0.1*rand(N)
ρ2 = apply_depo_channel(ρ1,p)
println("state prepared in the `experiment' 2 ")
data2 = zeros(Int8,(nu,NM,N))
@showprogress dt=1  for r in 1:nu
                data2[r,:,:] = get_RandomMeas(ρ2,u[r],NM)
end

state prepared in the `experiment' 2 


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


In [5]:
#Build histograms for each unitary
P1 = Vector{ITensor}()
P2 = Vector{ITensor}()
for r in 1:nu
                push!(P1,get_Born(data1[r,:,:],ξ))
                push!(P2,get_Born(data2[r,:,:],ξ))
end

In [6]:
#Uncomment if you want to see an animation
# using Plots
# anim = @animate for r in 1:nu
#     y1 =  reshape(array(P1[r]),(2^N))
#     y2 =  reshape(array(P2[r]),(2^N))
#     bar(y1,label="Experiment 1")
#     bar!(y2,label="Experiment 2")
#     xlabel!("bitstring index")
#     ylabel!("Probabilities")
#     title!("Random unitary $(r)")
# end
# gif(anim,fps=0.5)

In [9]:
overlaps = 0
for r in 1:nu
    global overlaps += get_overlap(P1[r],P2[r],ξ,N)/nu
end
purity1s = get_purity_hamming(data1,ξ)
purity2s = get_purity_hamming(data2,ξ)
Fs = overlaps/max(purity1s,purity2s)
println("Estimated overlap ",overlaps)
println("Estimated purity1 ",purity1s)
println("Estimated purity2 ",purity2s)
println("Estimated fidelity ",Fs)

Estimated overlap 0.7522639899999999
Estimated purity1 0.9639698498498492
Estimated purity2 0.5937543543543541
Estimated fidelity 0.7803812433730938


In [8]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("overlap ",overlap)
println("purity1 ",purity1)
println("purity2 ",purity2)
println("fidelity ",F)

overlap 0.7879369592497474
purity1 1.0
purity2 0.6262698620021796
fidelity 0.7879369592497474
